# Project 3 for CSE142
## CIFAR-10 Classification

Importing necessary packages:

- tensorflow/keras: API to build and train models
- matplotlib: generate graphs
- numpy: perform array operations
- pandas: to work with dataframes
- pickle: for serializing and de-serializing a Python object
- os: for file paths

In [5]:
import tensorflow as tf#from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import matplotlib.image as mpimg 
import numpy as np
import pandas as pd
# import pickle
# import os, os.path

### *Initialize relevant variables*

In [7]:
labels_df = pd.read_csv("Semi1_Labels.csv") #change this for different versions
train_path = 'Train_Image' 
test_path = 'Test_Image' 
labels = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
labels_df

,Train Label
0,NAN
1,dog
2,NAN
3,airplane
4,NAN
5,NAN
6,NAN
7,NAN
8,deer
9,NAN


## Defining some functions

In [14]:
def get_image(img_number):
    img = mpimg.imread(train_path + f"/{img_number}.png") 
    return img
def get_test_image(img_number):
    img = mpimg.imread(test_path + f"/{img_number}.png") 
    return img

In [15]:
plt.imshow(get_image(7777))

FileNotFoundError: [Errno 2] No such file or directory: 'Train_Image/7777.png'

In [16]:
plt.imshow(get_test_image(999))

FileNotFoundError: [Errno 2] No such file or directory: 'Test_Image/999.png'

**Function that prints image info**

In [ ]:
def get_img_info(img):
    rows,cols,channels = img.shape 
    img_size = rows*cols*channels # channels means rgb colors i think, so should always be 3?
    img_to_1D = img.reshape(img_size) #numPy reshape
    print("rows, cols, channels:", rows, cols, channels)
    print("image size:", img_size)
#     print("image original:", img)
#     print("image reshape:", img_to_1D)
#     return img_to_1D

In [ ]:
get_img_info(get_image(3)) 

## Preprocessing the  Data

We need to separate the labeled data from the unlabeled

Also need to encode labels as numbers instead of strings

In [19]:
#is_unlabelled = labels_df["Train Label"] == NAN
labelled = (labels_df["Train Label"]!= "NAN")
unlabelled = (labels_df["Train Label"] == "NAN")

unlabelled_data = (labels_df[unlabelled])
print(unlabelled_data)

labelled_data = (labels_df[labelled])
print(labelled_data)

     Train Label
0            NAN
2            NAN
4            NAN
5            NAN
6            NAN
7            NAN
9            NAN
10           NAN
11           NAN
12           NAN
13           NAN
14           NAN
15           NAN
16           NAN
17           NAN
19           NAN
20           NAN
21           NAN
22           NAN
23           NAN
24           NAN
25           NAN
28           NAN
31           NAN
34           NAN
35           NAN
36           NAN
38           NAN
39           NAN
40           NAN
...          ...
9959         NAN
9960         NAN
9961         NAN
9962         NAN
9963         NAN
9966         NAN
9967         NAN
9968         NAN
9971         NAN
9972         NAN
9973         NAN
9975         NAN
9978         NAN
9979         NAN
9980         NAN
9981         NAN
9982         NAN
9983         NAN
9984         NAN
9985         NAN
9986         NAN
9987         NAN
9989         NAN
9991         NAN
9993         NAN
9994         NAN
9995         N

In [23]:
labeled = []
unlabeled = []

labeled = labelled_data["Train Label"].tolist()
unlabeled = unlabelled_data["Train Label"].tolist()

print (labeled) 
print (unlabeled) 

['dog', 'airplane', 'deer', 'frog', 'deer', 'dog', 'cat', 'airplane', 'bird', 'automobile', 'cat', 'automobile', 'dog', 'ship', 'cat', 'deer', 'cat', 'deer', 'bird', 'frog', 'frog', 'horse', 'frog', 'frog', 'ship', 'cat', 'truck', 'bird', 'bird', 'horse', 'deer', 'dog', 'cat', 'horse', 'airplane', 'dog', 'truck', 'airplane', 'cat', 'frog', 'ship', 'horse', 'horse', 'truck', 'truck', 'airplane', 'truck', 'truck', 'cat', 'horse', 'frog', 'frog', 'deer', 'automobile', 'dog', 'dog', 'cat', 'horse', 'frog', 'automobile', 'deer', 'ship', 'dog', 'bird', 'airplane', 'horse', 'airplane', 'cat', 'airplane', 'automobile', 'deer', 'frog', 'ship', 'frog', 'ship', 'dog', 'horse', 'dog', 'horse', 'bird', 'airplane', 'dog', 'frog', 'automobile', 'automobile', 'truck', 'ship', 'frog', 'horse', 'frog', 'ship', 'deer', 'automobile', 'frog', 'automobile', 'deer', 'horse', 'cat', 'truck', 'horse', 'cat', 'automobile', 'bird', 'truck', 'truck', 'automobile', 'bird', 'airplane', 'automobile', 'frog', 'horse'

In [20]:
#col = labels_df.columns
#for i, entry in enumerate(labels_df[col[0]]):
 #   if (entry == 'NAN'):
  #      unlabeled.append(i)
   # else:
   #     labeled.append([i, entry])
# print("labeled", labeled)
# print("unlabeled", unlabeled)

IndentationError: unexpected indent (<ipython-input-20-6c372edd5685>, line 6)

In [ ]:
# Need to reshape training data and labels
train_images, train_labels, test_images, test_labels = [], [], [], []
for i in range(len(labeled)):
    train_images.append(get_image(labeled[i][0]))
    train_labels.append(labeled[i][1])
print(len(train_images))
for i in range(36):
    plt.subplot(6,6,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()
    plt.grid(False)
    plt.imshow(train_images[i])
    plt.xlabel(train_labels[i])
plt.show()

In [ ]:
#encoding labels
for i in range(len(train_labels)):
    for j in range(len(labels)):
        if train_labels[i] == labels[j]:
            train_labels[i] = j
print (train_labels[1])

In [ ]:
#split labeled data so that we have some labeled testing data 80/20
test_images =  np.array(train_images[1600:])
train_images = np.array(train_images[:1600])
test_labels = np.array(train_labels[1600:])
train_labels = np.array(train_labels[:1600])
print(len(train_images), len(train_labels), len(test_images), len(test_labels))


## Baseline Model

### Convolutional Neural Network

As input, a CNN takes tensors of shape (height, width, channels) = (32, 32, 3)

This is based on https://www.tensorflow.org/tutorials/images/cnn

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))

## Augmenting the Model

